<a href="https://www.kaggle.com/code/ironman45/mask-detection?scriptVersionId=162817934" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
os.listdir('/kaggle/input')

In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization 
from tensorflow.keras.models import Sequential 
import matplotlib.pyplot as plt

data=tf.keras.utils.image_dataset_from_directory("/kaggle/input/face-mask-dataset/data",batch_size=32) 
data=data.map(lambda x,y:(x/255.0,y))

data_i=data.as_numpy_iterator()

# Preparing the Dataset for training and validation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch_size=32

Train_Datagen = ImageDataGenerator(dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)
Val_Datagen = ImageDataGenerator(dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)

train_gen = Train_Datagen.flow_from_directory(directory = '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train', target_size = (256,256), 
                                       batch_size = batch_size, class_mode = 'binary')

val_gen = Val_Datagen.flow_from_directory(directory = '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation', target_size = (256,256), 
                                       batch_size = batch_size, class_mode = 'binary')

# Building The CNN model for Mask Detection

In [ ]:
model=Sequential()

pretrained_model=tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    
    input_shape=(256,256,3),
    pooling='avg',
)

for layer in pretrained_model.layers:
    layer.trainable=False

model.add(pretrained_model)

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))

model.compile('adam',loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])


In [ ]:
model.summary()

# Training the model

In [ ]:
hist=model.fit(train_gen,epochs=15,validation_data=val_gen)

# Plot of Training Loss vs Validation Loss

In [ ]:
plt.plot(hist.history['loss'],color='red',label='loss');
plt.plot(hist.history['val_loss'],color='green',label='val_loss');


# Plot of Training Accuracy vs Validation Accuracy

In [ ]:
plt.plot(hist.history['accuracy'],color='red',label='loss');
plt.plot(hist.history['val_accuracy'],color='green',label='val_loss');


In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
pre=Precision()
re=Recall()
acc=BinaryAccuracy()

# Generating Test Data

In [ ]:
Test_Datagen = ImageDataGenerator(dtype = 'float32', preprocessing_function=tf.keras.applications.resnet.preprocess_input)

test_gen = Test_Datagen.flow_from_directory(directory = '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test', target_size = (256,256), 
                                       batch_size = 32, class_mode = 'binary')



In [ ]:
for i in range(30):
    batch=test_gen[i]
    x,y=batch
    ypred=model.predict(x)
    pre.update_state(y,ypred)
    re.update_state(y,ypred)
    acc.update_state(y,ypred)
    

# Model Accuracy

In [ ]:
print(pre.result().numpy())
print(re.result().numpy())
print(acc.result().numpy())

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
image=data.as_numpy_iterator().next()
X,y=image

fig = plt.figure(figsize=(32,32)) 


for i in range(32):
    fig.add_subplot(8, 4, i+1) 

    plt.imshow((X[i]*255).astype(int))

    my_image = img_to_array(X[i]*255)
    my_image = my_image.reshape((1, my_image.shape[0], my_image.shape[1], my_image.shape[2]))
    my_image = preprocess_input(my_image)



    pre=model.predict(my_image)
    if(pre<0.5):
        plt.title("MASK ON",color='green') 
    else:
        plt.title("NO MASK",color='red') 

